In [2]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict
import multiprocessing as mp
import glob
import sys, os
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import gezi
from gezi import tqdm
tqdm.pandas()
import sentencepiece as spm
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

2021-10-12 15:59:13.745306: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
d = pd.read_csv('../input/info/infos.csv')

In [ ]:
with open('../input/corpus.txt', 'w') as f:
  for row in tqdm(d.itertuples(), total=len(d)):
    print(row.title, file=f)
    print(row.asr_text, file=f)

In [ ]:
spm.SentencePieceTrainer.train(input='../input/corpus.txt', model_prefix='../input/sp10w', vocab_size=100000, user_defined_symbols=[f'SEP{i}' for i in range(200)])

In [6]:
sp = spm.SentencePieceProcessor(model_file='../input/sp10w.model')
sp.encode('我爱你中国')

[4813, 460]

In [7]:
sp.decode([4813, 460])

'我爱你中国'

In [ ]:
for line in open('../input/corpus.txt'):
  line = line.strip()
  print(line)
  print(sp.encode(line))
  print(sp.encode(line, out_type=str))
  break

In [ ]:
vocab = gezi.Vocab('../input/sp10w.vocab', 0)

In [ ]:
sp.encode('我只不过')

In [ ]:
vocab.id('我')

In [ ]:
vocab.id('光子')

In [ ]:
vocab.key(101)

In [ ]:
vocab_names = [
                'tag',
                'word',
              ]
vocabs = {}
for vocab_name in vocab_names:
  if vocab_name != 'word':
    vocab_file =  f'../input/{vocab_name}_vocab.txt'
    vocab = gezi.Vocab(vocab_file)
  else:
    vocab_file = '../input/sp10w.vocab'
    vocab = gezi.Vocab(vocab_file, 0)
  vocabs[vocab_name] = vocab

In [ ]:
d.head()

In [ ]:
title_words = []
asr_words = []
for row in tqdm(d.itertuples(), total=len(d)):
  title_words.append(len(sp.encode(row.title)))
  asr_words.append(len(sp.encode(str(row.asr_text))))

In [ ]:
d['title_words'] = title_words
d['asr_words'] = asr_words

In [ ]:
d.title_words.describe([.5,.9,.99,.999])

In [ ]:
d.asr_words.describe([.5,.9,.99,.999])

In [ ]:
def gen_w2v(window=32, min_count=1, emb_dim=256):
  sentences = []
  for row in tqdm(d.itertuples(), total=len(d)):
    row = row._asdict()
    l = ['[CLS]', *sp.encode(str(row['title']), out_type=str), '[SEP]', *sp.encode(str(row['asr_text']), out_type=str), '[SEP]']
    sentences.append(l)
  ic(len(sentences))
  name = 'word'
  monitor = gezi.MonitorCallback(name) 
  w2v = Word2Vec(sentences, vector_size=emb_dim, window=window, min_count=min_count, sg=1, workers=mp.cpu_count(), epochs=10, callbacks=[monitor])
  ofile = f'../input/w2v/sp/{emb_dim}/{name}.pkl'
  gezi.try_mkdir(os.path.dirname(ofile))
  gezi.save_pickle(w2v, ofile)
  for name in vocabs:
    vocab = vocabs[name]
    # emb = np.zeros([vocab.size(), emb_dim])
    emb = np.random.uniform(-0.05, 0.05,(vocab.size(), emb_dim))
    for i in range(vocab.size()):
      word = vocab.key(i) 
      if word in w2v.wv:
        emb[i] = w2v.wv[word]
    ofile = f'../input/w2v/sp/{emb_dim}/{name}.npy'
    np.save(ofile, emb)
  
  return w2v

In [47]:
# gen_w2v(emb_dim=256)

In [ ]:
# gen_w2v(emb_dim=512)

In [ ]:
gen_w2v(emb_dim=768)